In [1]:
#Imports
import autograd.numpy as np
from autograd import grad
import pandas as pd
from scipy.optimize import curve_fit, minimize
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from itertools import product
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import CLD_CONTINUED
import warnings
import numpy as np
from scipy.optimize import basinhopping
from sklearn.metrics._plot.confusion_matrix import unique_labels

from scipy.optimize import OptimizeWarning
np.seterr(over='ignore')
np.seterr(invalid='ignore')
warnings.filterwarnings("ignore", category=OptimizeWarning)

In [2]:
DEFAULT_PROJECT_DIR = "/fsx-onellm/margaretli/env_srcs/xlf/xlformers_n/scaling/data"

col_names = ['C', 'D', 'N', 'lr', 'Avg Train Loss', 'Max Train Loss', 'C4 Eval PPL', 'Wiki Eval PPL', 'C4 Eval Loss', 'Wiki Eval Loss']

def read_data(csv_file, loss_name='C4 Eval Loss', col_names=col_names):
    mins_only = []
    df = pd.read_csv(csv_file, usecols=col_names,)
    df.dropna(subset=[loss_name], inplace=True)

    if 'lr' in df.columns:
        df = df.loc[(df['lr'] >= 0)]
    if 'D' not in df.columns:
        df['D'] = df['C'] / (df['N'] * 6)
        
    n_vals = df['N'].unique()
    d_vals = sorted(df['D'].unique())
    for n in n_vals:
        for d in d_vals:
            cd_df = df[(df['N'] == n) & (df['D'] == d)]
            if cd_df.empty:
                continue
            min_index = cd_df[loss_name].idxmin()
            # print(min_index)
            # print(cd_df)
            # print(cd_df.loc[min_index])
            mins_only.append(cd_df.loc[min_index])

    mins_only_df = pd.DataFrame(mins_only)

    print(mins_only_df)
    # df.rename(columns={})
    return mins_only_df

In [3]:
def scaling_law(N, D, params):
    if len(params) == 6:
        params = params[:-1]
    a, b, e, alpha, beta = params
        
    A = np.exp(a)
    B = np.exp(b)
    E = np.exp(e)
    
    L = E + (A / (N**alpha)) + (B /(D**beta))
    
    return L

def opt_N_D(C, G, opt_a, opt_b):
    opt_N = G*(C/6)**opt_a
    opt_D = (1/G)*(C/6)**opt_b
    return opt_N, opt_D

def print_opts(best_params):
    if len(best_params) == 6:
        best_params = best_params[:-1]
    opt_alpha = best_params[-2]
    opt_beta = best_params[-1]

    opt_a =  opt_beta / (opt_alpha+opt_beta)
    opt_b =  opt_alpha / (opt_alpha+opt_beta)

    A = np.exp(best_params[0])
    B = np.exp(best_params[1])
    G = ((opt_alpha*A)/(opt_beta*B))**(1/(opt_alpha+opt_beta))

    scaling = []

    for C in [1.25E+18, 5.01E+18, 1.98E+19, 1E21, 1E23]:
        N, D = opt_N_D(C, G, opt_a, opt_b)
        scaling.append(
            {"compute": f"{C:e}",
            "parameters (B)": f"{N/1e9:.2f}",
            "tokens (B)": f"{D/1e9:.2f}",
            "ratio": f"{D/N:.2f}",
            "predicted loss": f"{scaling_law(N, D, best_params):.2f}"
            }
        )
    print("Scaling: ", pd.DataFrame(scaling))
        

In [18]:
loss_name = ""
csv_file = "/fsx-onellm/margaretli/env_srcs/xlf/xlformers_n/scaling/data/data.csv"

training_df = read_data(csv_file=csv_file, loss_name=loss_name, col_names=None)

N = training_df['N'].values
D = training_df['D'].values
losses = training_df[loss_name].values
bootstraps = 4000
nr_of_models_excluded = 5

sorted_losses = sorted(losses)
if nr_of_models_excluded == 0:
    indices = list(range(len(N)))
else:
    sorted_losses = sorted(losses)
    indices = [i for i in range(len(N)) if losses[i] < sorted_losses[-nr_of_models_excluded]]

np.random.seed(42)
random_indices = [np.random.choice(indices, size=len(indices), replace=True) for _ in range(bootstraps)]

KeyError: ['loss']

In [5]:
import autograd.numpy as np
from autograd.scipy.stats import norm
from scipy.optimize import minimize
from scipy.special import erf

true_params = np.array([np.log(406.4), np.log(410.7), np.log(1.69), 0.34, 0.28])

# Define the log-sum-exp function
def log_sum_exp(a, b, e, alpha, beta, N, D):
    return np.log(np.exp(a - alpha * np.log(N)) + np.exp(b - beta * np.log(D)) + np.exp(e))

# Define the Huber loss function
def custom_huber_loss(y_true, y_pred, delta=1e-3):
    # Calculate the difference
    diff = y_true - y_pred
    # Calculate the condition for Huber loss
    cond = np.abs(diff) <= delta
    # Apply Huber loss formula
    loss = np.where(cond, 0.5 * diff**2, delta * (np.abs(diff) - 0.5 * delta))
    return np.sum(loss)

def huber_normalizing_factor(delta=1e-3):
    return np.sqrt(2*np.pi) * (1 - 2*norm.sf(delta)) + 2 * np.exp(-0.5*delta**2)/delta

def huber_logpdf(x, delta=1e-3, loc=0, scale=1):
    x = (x-loc)/scale

    cond = np.abs(x) <= delta
    loss = np.where(cond, 0.5 * x**2, delta * (np.abs(x) - 0.5 * delta))
    return -loss - np.log(huber_normalizing_factor(delta=delta)) - np.log(scale)

def huber_pdf(x, delta=1e-3, loc=0, scale=1):
    return np.exp(huber_logpdf(x, delta=delta, loc=loc, scale=scale))

# Define the objective function to be minimized
def objective(params, N, D, losses):
    a, b, e, alpha, beta, sigma = params
    predictions = log_sum_exp(a, b, e, alpha, beta, N, D)
    return -np.sum(huber_logpdf(np.log(losses), loc=predictions, scale=np.exp(sigma), delta=1e-3))
    # return custom_huber_loss(np.log(losses), predictions, delta=1e-3)

def scale_objective(sigma, params, N, D, losses):
    a, b, e, alpha, beta = params
    predictions = log_sum_exp(a, b, e, alpha, beta, N, D)
    return -np.sum(huber_logpdf(np.log(losses), loc=predictions, scale=np.exp(sigma), delta=1e-3))
    # return custom_huber_loss(np.log(losses), predictions, delta=1e-3)

def constant_term_objective(params, a, b, alpha, beta, N, D, losses):
    e, sigma = params
    predictions = log_sum_exp(a, b, e, alpha, beta, N, D)
    return -np.sum(huber_logpdf(np.log(losses), loc=predictions, scale=np.exp(sigma), delta=1e-3))

def huber_loss_objective(params, N, D, losses):
    a, b, e, alpha, beta = params
    predictions = log_sum_exp(a, b, e, alpha, beta, N, D)
    return custom_huber_loss(np.log(losses), predictions, delta=1e-3)

# Define the parameter untransform
def untransform_params(param_array):
    if len(np.shape(param_array)) == 2:
      return np.hstack((np.exp(param_array[:, :3]), param_array[:, 3:]))
    else:
      return np.hstack((np.exp(param_array[:3]), param_array[3:]))

# Define the Huber loss function on residuals
def huber_loss(residuals, delta=1e-3):
    # Calculate the difference
    diff = residuals
    # Calculate the condition for Huber loss
    cond = np.abs(diff) <= delta
    # Apply Huber loss formula
    loss = np.where(cond, 0.5 * diff**2, delta * (np.abs(diff) - 0.5 * delta))
    return loss

In [14]:
def fit_from_scratch(obj=huber_loss_objective, method='BFGS', use_grad=True, add_sigma=False):
    # Set up the grid for initial parameter values
    alpha_vals = np.arange(0, 2.5, 0.5)
    beta_vals = np.arange(0, 2.5, 0.5)
    e_vals = np.arange(-1, 1.5, 0.5)
    a_vals = np.arange(0, 30, 5)
    b_vals = np.arange(0, 30, 5)

    # Perform the optimization using L-BFGS over the grid of initial values
    best_loss = np.inf
    best_params = None

    from itertools import product
    results_dict = {}
    for alpha, beta, e, a, b in product(alpha_vals, beta_vals, e_vals, a_vals, b_vals):
        init_params = [a, b, e, alpha, beta]
        if add_sigma:
            init_params = init_params + [0]
        result = minimize(obj, init_params, args=(N[indices], D[indices], losses[indices]), method=method, jac=grad(obj) if use_grad else None)
        results_dict[tuple(init_params)] = {'params': result.x, 'loss': result.fun}
        if result.success and result.fun < best_loss:
            best_loss = result.fun
            best_params = result.x
            print(f"New best loss: {best_loss}")
            print(f"Best params: {best_params}")
            print(f"Initial guess: {init_params}")

    # Transform the fitted parameters a, b, e to A, B, E
    if best_params is not None:
        A = np.exp(best_params[0])
        B = np.exp(best_params[1])
        E = np.exp(best_params[2])
        alpha = best_params[3]
        beta = best_params[4]
        print(f"Best fit parameters: A={A}, B={B}, E={E}, alpha={alpha}, beta={beta}")
        print_opts(best_params)
    else:
        print("Optimization failed to converge.")

def fit_from_chinchill_random(obj=huber_loss_objective, method='BFGS', use_grad=True, add_sigma=False):
    # Set up the grid for initial parameter values
    param_list = []

    for num, indices in enumerate(random_indices):
    # Perform the optimization using BFGS
        best_loss = np.inf
        best_params = None

        init_params = list(true_params)
            
        if add_sigma:
            init_params = init_params + [0]

        result = minimize(obj, init_params, args=(N[indices], D[indices], losses[indices]), \
                            jac=grad(obj) if use_grad else None, method=method)

        best_loss = result.fun
        best_params = result.x
        #print(f"New best loss: {best_loss}")
        #print(f"Best params: {best_params}")

        if num % 1000 == 999:
            print("Bootstrap step %d completed" % (num+1))

        param_list.append(result.x)

    param_list = np.array(param_list)
    cov_matrix = np.cov(np.transpose(param_list))

    if best_params is not None:
        A = np.exp(best_params[0])
        B = np.exp(best_params[1])
        E = np.exp(best_params[2])
        alpha = best_params[3]
        beta = best_params[4]
        print(f"Best fit parameters: A={A}, B={B}, E={E}, alpha={alpha}, beta={beta}")
        print_opts(best_params)
    else:
        print("Optimization failed to converge.")

def fit_from_chinchilla(obj=huber_loss_objective, method='BFGS', use_grad=True, add_sigma=False):

    init_params = list(true_params)
    
    if add_sigma:
        init_params = init_params + [0]

    indices = list(range(len(N))) if nr_of_models_excluded == 0 else [i for i in range(len(N)) if losses[i] < sorted(losses)[-nr_of_models_excluded]]

    result = minimize(obj, init_params, args=(N[indices], D[indices], losses[indices]), method=method, jac=grad(obj) if use_grad else None)

    print(result)
    print(result.x)

    estimated_params = result.x[:5]
    # best_params = untransform_params(estimated_params)
    best_params = estimated_params
    if best_params is not None:
        A = np.exp(best_params[0])
        B = np.exp(best_params[1])
        E = np.exp(best_params[2])
        alpha = best_params[3]
        beta = best_params[4]
        print(f"Best fit parameters: A={A}, B={B}, E={E}, alpha={alpha}, beta={beta}")
        print_opts(best_params)
    else:
        print("Optimization failed to converge.")

In [7]:
fit_from_scratch(obj=huber_loss_objective, method='L-BFGS-B', use_grad=False)

New best loss: 0.0011053473780280002
Best params: [6.75255148 6.5769012  0.58029286 0.38141966 0.31312923]
Initial guess: [0, 0, -1.0, 0.0, 0.0]
New best loss: 0.0010182741179027332
Best params: [6.16845887 7.66950834 0.59725681 0.34726469 0.36714071]
Initial guess: [0, 20, -1.0, 0.0, 0.0]
New best loss: 0.0010182741126764361
Best params: [6.16935134 7.66947527 0.59730042 0.34731918 0.3671382 ]
Initial guess: [10, 0, -0.5, 0.0, 0.0]
New best loss: 0.0010182740628076144
Best params: [6.16893451 7.66973506 0.59728104 0.34729221 0.36715146]
Initial guess: [10, 25, -0.5, 0.0, 0.0]
New best loss: 0.0010182740529862638
Best params: [6.1694601  7.66957911 0.59729565 0.34732307 0.36714366]
Initial guess: [25, 25, -0.5, 0.0, 0.5]
New best loss: 0.0010182740284042362
Best params: [6.16899519 7.67006798 0.59729623 0.34729593 0.36716834]
Initial guess: [10, 5, 0.0, 0.0, 0.5]
New best loss: 0.0010182740261777178
Best params: [6.16939887 7.67000597 0.59730795 0.34731959 0.36716492]
Initial guess: [2

In [8]:
fit_from_scratch(obj=huber_loss_objective, method='L-BFGS-B', use_grad=True)

New best loss: 0.0011083041682976925
Best params: [6.75771895 6.55095816 0.57943629 0.38171406 0.31185198]
Initial guess: [0, 0, -1.0, 0.0, 0.0]
New best loss: 0.0010182740180545722
Best params: [6.16922099 7.67016579 0.59730553 0.34730898 0.36717282]
Initial guess: [5, 20, -1.0, 0.0, 0.0]


New best loss: 0.0010182740180024207
Best params: [6.16920468 7.67015162 0.59730499 0.347308   0.36717224]
Initial guess: [25, 10, 0.5, 0.0, 0.0]
New best loss: 0.0010182740179315408
Best params: [6.16921471 7.67016498 0.59730565 0.34730861 0.36717286]
Initial guess: [15, 20, 0.0, 0.0, 0.5]
New best loss: 0.0010182740178419198
Best params: [6.16925725 7.67013872 0.59730677 0.34731119 0.36717152]
Initial guess: [0, 15, 0.5, 0.5, 0.0]
Best fit parameters: A=477.83106455805387, B=2143.3787613710106, E=1.817218025740887, alpha=0.34731118646490367, beta=0.3671715167962982
Scaling:          compute parameters (B) tokens (B)  ratio predicted loss
0  1.250000e+18           0.09       2.32  25.75           3.42
1  5.010000e+18           0.18       4.55  24.78           3.07
2  1.980000e+19           0.37       8.87  23.85           2.80
3  1.000000e+21           2.79      59.70  21.38           2.30
4  1.000000e+23          29.76     559.99  18.82           2.03


In [25]:
fit_from_chinchilla_random(obj=huber_loss_objective, method='BFGS', use_grad=True)

KeyboardInterrupt: 

In [15]:
fit_from_chinchilla(obj=huber_loss_objective, method='BFGS', use_grad=True)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 0.001018274018458569
        x: [ 6.169e+00  7.670e+00  5.973e-01  3.473e-01  3.672e-01]
      nit: 48
      jac: [-3.431e-07 -5.219e-08 -1.024e-06  6.373e-06  1.836e-06]
 hess_inv: [[ 6.477e+03 -1.779e+03 ...  3.874e+02 -9.183e+01]
            [-1.779e+03  7.168e+03 ... -1.155e+02  3.571e+02]
            ...
            [ 3.874e+02 -1.155e+02 ...  2.322e+01 -5.947e+00]
            [-9.183e+01  3.571e+02 ... -5.947e+00  1.780e+01]]
     nfev: 58
     njev: 58
[6.16916562 7.67020393 0.59730455 0.34730572 0.36717474]
Best fit parameters: A=477.78728593297524, B=2143.518526512652, E=1.8172139818981852, alpha=0.34730571518035713, beta=0.36717474434751385
Scaling:          compute parameters (B) tokens (B)  ratio predicted loss
0  1.250000e+18           0.09       2.32  25.75           3.42
1  5.010000e+18           0.18       4.55  24.78           3.07
2  1.980000e+19           0.37       8.87  23.85  

In [16]:
fit_from_scratch(obj=objective, method='BFGS', add_sigma=True, use_grad=True)

New best loss: -176.68988180964567
Best params: [14.85482276 14.89604692  0.95434598  2.98180101  2.44521689 -9.33711031]
Initial guess: [15, 15, -1.0, 0.0, 0.0, 0]
New best loss: -329.96908147439643
Best params: [19.81277459  4.33166672  0.41017194  3.84922001  0.18087016 -9.97577364]
Initial guess: [20, 5, -1.0, 0.0, 0.0, 0]


/fsx-onellm/margaretli/miniforge3/envs/mlexp/lib/python3.9/site-packages/autograd/tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)
/fsx-onellm/margaretli/miniforge3/envs/mlexp/lib/python3.9/site-packages/autograd/numpy/numpy_vjps.py:78: RuntimeWarning: divide by zero encountered in scalar divide
  defvjp(anp.log,    lambda ans, x : lambda g: g / x)
/fsx-onellm/margaretli/miniforge3/envs/mlexp/lib/python3.9/site-packages/autograd/tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)
/fsx-onellm/margaretli/miniforge3/envs/mlexp/lib/python3.9/site-packages/autograd/numpy/numpy_vjps.py:78: RuntimeWarning: divide by zero encountered in scalar divide
  defvjp(anp.log,    lambda ans, x : lambda g: g / x)
/fsx-onellm/margaretli/miniforge3/envs/mlexp/lib/python3.9/site-packages/autograd/tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)
/fsx-onellm/margaretli/min

New best loss: -879.7731390602361
Best params: [  6.17795598   7.64273236   0.59711196   0.34781303   0.36585412
 -12.26662388]
Initial guess: [10, 5, 0.0, 0.0, 0.0, 0]


/fsx-onellm/margaretli/miniforge3/envs/mlexp/lib/python3.9/site-packages/autograd/tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)
/fsx-onellm/margaretli/miniforge3/envs/mlexp/lib/python3.9/site-packages/autograd/numpy/numpy_vjps.py:78: RuntimeWarning: divide by zero encountered in scalar divide
  defvjp(anp.log,    lambda ans, x : lambda g: g / x)
/fsx-onellm/margaretli/miniforge3/envs/mlexp/lib/python3.9/site-packages/autograd/tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)
/fsx-onellm/margaretli/miniforge3/envs/mlexp/lib/python3.9/site-packages/autograd/numpy/numpy_vjps.py:78: RuntimeWarning: divide by zero encountered in scalar divide
  defvjp(anp.log,    lambda ans, x : lambda g: g / x)
/fsx-onellm/margaretli/miniforge3/envs/mlexp/lib/python3.9/site-packages/autograd/tracer.py:48: RuntimeWarning: divide by zero encountered in log
  return f_raw(*args, **kwargs)
/fsx-onellm/margaretli/min

Best fit parameters: A=482.0057174102976, B=2085.4342005798085, E=1.8168640396457942, alpha=0.34781302903946176, beta=0.36585411729446576
Scaling:          compute parameters (B) tokens (B)  ratio predicted loss
0  1.250000e+18           0.09       2.30  25.50           3.42
1  5.010000e+18           0.18       4.53  24.62           3.07
2  1.980000e+19           0.37       8.86  23.78           2.80
3  1.000000e+21           2.78      59.91  21.53           2.30
4  1.000000e+23          29.49     565.19  19.17           2.03


In [ ]:
fit_from_chinchilla_random(obj=objective, method='BFGS', add_sigma=True, use_grad=True)

In [17]:
fit_from_chinchilla(obj=objective, method='BFGS', add_sigma=True, use_grad=True)

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: -879.7731390602354
        x: [ 6.178e+00  7.643e+00  5.971e-01  3.478e-01  3.659e-01
            -1.227e+01]
      nit: 60
      jac: [ 8.871e-03  4.238e-03  3.696e-02 -1.729e-01 -1.087e-01
             4.217e-06]
 hess_inv: [[ 7.170e-07 -4.260e-07 ... -2.225e-08 -5.241e-07]
            [-4.260e-07  2.267e-06 ...  1.127e-07  4.873e-06]
            ...
            [-2.225e-08  1.127e-07 ...  5.607e-09  2.366e-07]
            [-5.241e-07  4.873e-06 ...  2.366e-07  3.910e-03]]
     nfev: 202
     njev: 192
[  6.17795598   7.64273236   0.59711196   0.34781303   0.36585412
 -12.26662386]
Best fit parameters: A=482.0057173974339, B=2085.4342007291266, E=1.8168640396503732, alpha=0.3478130290378019, beta=0.365854117298256
Scaling:          compute parameters (B) tokens (B)  ratio predicted loss
0  1.250000e+18           0.09       2.30  25.50           3.42
1  5.010000e+18       